Original Paper: 

https://arxiv.org/abs/2108.12184

Original code and dataset: 

https://github.com/usydnlp/Glocal_K

One part of our project is the reconstruction is 
the reconstruction of the GLocal_K algorithm with a different approach: we replaced ALL of tensorflow codes with pytorch, which essentially means to redo all the key parts regarding to the model. Other than that, we basically follow the original ideas of the paper and structure of the existing code. 

In [470]:
from google.colab import drive
drive.mount('/content/drive')

from zipfile import ZipFile
file_name = '/content/data.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')
import warnings
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Done


In [471]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from time import time
from scipy.sparse import csc_matrix
import numpy as np
import h5py

# Data Loader Function

In [472]:
def load_data_100k(path='./', delimiter='\t'):

    train = np.loadtxt(path+'movielens_100k_u1.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'movielens_100k_u1.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:,0]).size  # num of users
    n_m = np.unique(total[:,1]).size  # num of movies
    n_train = train.shape[0]  # num of training ratings
    n_test = test.shape[0]  # num of test ratings

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_train):
        train_r[train[i,1]-1, train[i,0]-1] = train[i,2]

    for i in range(n_test):
        test_r[test[i,1]-1, test[i,0]-1] = test[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [473]:
def load_data_1m(path='./', delimiter='::', frac=0.1, seed=1234):

    tic = time()
    print('reading data...')
    data = np.loadtxt(path+'movielens_1m_dataset.dat', skiprows=0, delimiter=delimiter).astype('int32')
    print('taken', time() - tic, 'seconds')

    n_u = np.unique(data[:,0]).size  # num of users
    n_m = np.unique(data[:,1]).size  # num of movies
    n_r = data.shape[0]  # num of ratings

    udict = {}
    for i, u in enumerate(np.unique(data[:,0]).tolist()):
        udict[u] = i
    mdict = {}
    for i, m in enumerate(np.unique(data[:,1]).tolist()):
        mdict[m] = i

    np.random.seed(seed)
    idx = np.arange(n_r)
    np.random.shuffle(idx)

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_r):
        u_id = data[idx[i], 0]
        m_id = data[idx[i], 1]
        r = data[idx[i], 2]

        if i < int(frac * n_r):
            test_r[mdict[m_id], udict[u_id]] = r
        else:
            train_r[mdict[m_id], udict[u_id]] = r

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_r - int(frac * n_r)))
    print('num of test ratings: {}'.format(int(frac * n_r)))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [474]:
def load_matlab_file(path_file, name_field):
    
    db = h5py.File(path_file, 'r')
    ds = db[name_field]

    try:
        if 'ir' in ds.keys():
            data = np.asarray(ds['data'])
            ir   = np.asarray(ds['ir'])
            jc   = np.asarray(ds['jc'])
            out  = csc_matrix((data, ir, jc)).astype(np.float32)
    except AttributeError:
        out = np.asarray(ds).astype(np.float32).T

    db.close()

    return out

In [475]:
def load_data_monti(path='./'):

    M = load_matlab_file(path+'douban_monti_dataset.mat', 'M')
    Otraining = load_matlab_file(path+'douban_monti_dataset.mat', 'Otraining') * M
    Otest = load_matlab_file(path+'douban_monti_dataset.mat', 'Otest') * M

    n_u = M.shape[0]  # num of users
    n_m = M.shape[1]  # num of movies
    n_train = Otraining[np.where(Otraining)].size  # num of training ratings
    n_test = Otest[np.where(Otest)].size  # num of test ratings

    train_r = Otraining.T
    test_r = Otest.T

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

# Load Data

In [476]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
data_path = '/content/data'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [477]:
# Select a dataset among 'ML-1M', 'ML-100K', and 'Douban'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
dataset = 'ML-100K'
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [478]:
# Data Load
try:
    if dataset == 'ML-100K':
        path = data_path + '/MovieLens_100K/'
        n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(path=path, delimiter='\t')

    elif dataset == 'ML-1M':
        path = data_path + '/MovieLens_1M/'
        n_m, n_u, train_r, train_m, test_r, test_m = load_data_1m(path=path, delimiter='::', frac=0.1, seed=1234)

    elif dataset == 'Douban':
        path = data_path + '/Douban_monti/'
        n_m, n_u, train_r, train_m, test_r, test_m = load_data_monti(path=path)

    else:
        raise ValueError

except ValueError:
    print('Error: Unable to load data')

# Dim of data
print("train_r: ", train_r.shape)
print("train_m: ", train_m.shape)
print("test_r: ", test_r.shape)
print("test_m: ", test_m.shape)
print("n_m: ", n_m)
print("n_u: ", n_u)

data matrix loaded
num of users: 943
num of movies: 1682
num of training ratings: 80000
num of test ratings: 20000
train_r:  (1682, 943)
train_m:  (1682, 943)
test_r:  (1682, 943)
test_m:  (1682, 943)
n_m:  1682
n_u:  943


# Hyperparameter Settings

In [479]:
# Common hyperparameter settings
n_hid = 500
n_dim = 5
n_layers = 2
gk_size = 3

In [480]:
# Different hyperparameter settings for each dataset
if dataset == 'ML-100K':
    lambda_2 = 20.  # l2 regularisation
    lambda_s = 0.006
    # iter_p = 5  # optimisation
    # iter_f = 5
    epoch_p = 30  # training epoch
    epoch_f = 60
    dot_scale = 1  # scaled dot product

elif dataset == 'ML-1M':
    lambda_2 = 70.
    lambda_s = 0.018
    # iter_p = 50
    # iter_f = 10
    epoch_p = 20
    epoch_f = 30
    dot_scale = 0.5

elif dataset == 'Douban':
    lambda_2 = 10.
    lambda_s = 0.022
    # iter_p = 5
    # iter_f = 5
    epoch_p = 20
    epoch_f = 60
    dot_scale = 2

In [481]:
input_shape = (n_m, n_u)

# Create an input tensor
R = torch.zeros(input_shape, dtype=torch.float32, requires_grad=True)
# print(R.shape)

# Network Function

## Local Kernel:

In [482]:
def local_kernel(u, v):
    dist = torch.norm(u - v, dim=2, p=2)
    hat = torch.clamp(1 - dist**2, min=0)
    return hat

## Global Kernel

In [483]:
def global_kernel(input, gk_size, dot_scale):
    avg_pooling = torch.mean(input, dim=1).unsqueeze(0)
    n_kernel = avg_pooling.size(1)

    conv_kernel = nn.Parameter(torch.empty(n_kernel, gk_size**2).normal_(0, 0.1))
    gk = torch.matmul(avg_pooling, conv_kernel) * dot_scale
    gk = gk.view(gk_size, gk_size, 1, 1)

    return gk

import  tensorflow as tf

def global_conv(input, W):

    input = torch.reshape(input, [1, 1, input.shape[0], input.shape[1]])
    conv2d = torch.nn.functional.relu(torch.nn.functional.conv2d(input, W.detach().numpy(), stride=(1,1), padding=(1,1)))

    return torch.reshape(conv2d, [conv2d.shape[2], conv2d.shape[3]])

In [484]:
class KernelLayer(nn.Module):
    def __init__(self, input_shape, n_hid, n_dim, activation, lambda_s=1e-4, lambda_2=1e-4, name=''):
        super(KernelLayer, self).__init__()

        self.W = nn.Parameter(torch.empty(input_shape[1], n_hid))
        nn.init.xavier_uniform_(self.W)

        self.u = nn.Parameter(torch.empty(input_shape[1], 1, n_dim).normal_(0, 1e-3))
        self.v = nn.Parameter(torch.empty(1, n_hid, n_dim).normal_(0, 1e-3))
        self.b = nn.Parameter(torch.zeros(n_hid))

        self.activation = activation
        self.lambda_s = lambda_s
        self.lambda_2 = lambda_2

    def forward(self, x):

        w_hat = local_kernel(self.u, self.v)
        # print("x.shape", x.shape)
        sparse_reg_term = torch.norm(w_hat, p=2) * self.lambda_s
        l2_reg_term = torch.norm(self.W, p=2) * self.lambda_2
        
        W_eff = self.W * w_hat  # Local kernelised weight matrix
        y = torch.matmul(x, W_eff) + self.b
        y = self.activation(y)
        return y, sparse_reg_term + l2_reg_term

# Network Instantiation

## Pre-training
One thing to notice here is that I did not implement this part (and following parts) as what presented in the original code base.

Instead, I used a typical method to implement ***model*** using ***class*** as what we normally do using pytorch and tensorflow 2.x (the original work used 1.x).

In [485]:
class KernelModel(nn.Module):
    def __init__(self, n_layers, input_shape, n_hid, n_dim, n_u):
        super(KernelModel, self).__init__()
        self.layers = nn.ModuleList()
        for _ in range(n_layers):
            layer = KernelLayer(input_shape, n_hid, n_dim, torch.sigmoid)
            self.layers.append(layer)
            input_shape = (input_shape[0], n_hid)  # Update input_shape for the next layer
        self.output_layer = KernelLayer(input_shape, n_u, n_dim, activation=lambda x: x)

    def forward(self, x):
        reg_losses = 0
        for layer in self.layers:
            x, reg_loss = layer(x)
            reg_losses += reg_loss.item()
        pred, reg_loss = self.output_layer(x)
        reg_losses += reg_loss.item()
        return pred, reg_losses


y = R
reg_losses = 0
model_p = KernelModel(n_layers, y.shape, n_hid, n_dim, n_u)
pred_d, reg_losses = model_p(y)
print(reg_losses)

# Compute L2 loss (Which is equivalent to mse loss)

train_r = torch.tensor(train_r)
train_m = torch.tensor(train_m)

diff = train_m * (train_r - pred_d)

sqE = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff), reduction='sum')
loss_p = sqE + reg_losses
print(loss_p)

# Use Adam optimizer to minimize loss
optimizer_p = optim.Adam(model_p.parameters(), lr=0.05)


0.19468190521001816
tensor(1104852.2500, grad_fn=<AddBackward0>)


## Fine-tuning

In [486]:
class KernelModelF(nn.Module):
    def __init__(self, n_layers, input_shape, n_hid, n_dim, n_u, gk_size, dot_scale):
        super(KernelModelF, self).__init__()
        self.layers1 = nn.ModuleList()
        self.layers2 = nn.ModuleList()
        for _ in range(n_layers):
            layer = KernelLayer(input_shape, n_hid, n_dim, torch.sigmoid)
            self.layers.append(layer)
            input_shape = (input_shape[0], n_hid)  # Update input_shape for the next layer
        # for _ in range(n_layers):
        #     layer = KernelLayer(input_shape, n_hid, n_dim, torch.sigmoid)
        #     self.layers2.append(layer)
        #     input_shape = (input_shape[0], n_hid)  # Update input_shape for the next layer
        self.output_layer = KernelLayer(input_shape, n_u, n_dim, activation=lambda x: x)
        self.gk_size = gk_size
        self.dot_scale = dot_scale

    def forward(self, x):
        reg_losses = None
        for layer in self.layers:
            x, reg_loss = layer(x)

        y_dash, _ = self.output_layer(x)


        gk = global_kernel(y_dash, self.gk_size, self.dot_scale)

        y_hat = global_conv(train_r, gk)
        y_hat = torch.tensor(np.array(y_hat))

        for layer in self.layers:
            y_hat, reg_loss = layer(y_hat)
            reg_losses = reg_loss if reg_losses is None else reg_losses + reg_loss
        pred_f, reg_loss = self.output_layer(y_hat)
        reg_losses += reg_loss.item()

        return pred_f, reg_losses



y = R
reg_losses = None
model_f = KernelModel(n_layers, y.shape, n_hid, n_dim, n_u)
pred_f, reg_losses = model_f(y)
print(reg_losses)

# Compute L2 loss

train_r = torch.tensor(train_r)
train_m = torch.tensor(train_m)

diff_f = train_m * (train_r - pred_f)

sqE = torch.nn.functional.mse_loss(diff_f, torch.zeros_like(diff_f), reduction='sum')
loss_f = sqE + reg_losses
print(loss_f)

optimizer_f = optim.Adam(model_f.parameters(), lr=0.05)



0.19467797875404358
tensor(1105990.8750, grad_fn=<AddBackward0>)


# Evaluation code

In [487]:
def dcg_k(score_label, k):
    dcg, i = 0., 0
    for s in score_label:
        if i < k:
            dcg += (2**s[1]-1) / np.log2(2+i)
            i += 1
    return dcg

In [488]:
def ndcg_k(y_hat, y, k):
    score_label = np.stack([y_hat, y], axis=1).tolist()
    score_label = sorted(score_label, key=lambda d:d[0], reverse=True)
    score_label_ = sorted(score_label, key=lambda d:d[1], reverse=True)
    norm, i = 0., 0
    for s in score_label_:
        if i < k:
            norm += (2**s[1]-1) / np.log2(2+i)
            i += 1
    dcg = dcg_k(score_label, k)
    return dcg / norm

In [489]:
def call_ndcg(y_hat, y):
    ndcg_sum, num = 0, 0
    y_hat, y = y_hat.T, y.T
    n_users = y.shape[0]

    for i in range(n_users):
        y_hat_i = y_hat[i][np.where(y[i])]
        y_i = y[i][np.where(y[i])]

        if y_i.shape[0] < 2:
            continue

        ndcg_sum += ndcg_k(y_hat_i, y_i, y_i.shape[0])  # user-wise calculation
        num += 1

    return ndcg_sum / num

# Training and Test Loop

## Helper Functions:

In [490]:
def closure_p():
    optimizer_p.zero_grad()
    pred, reg_losses = model_p(train_r)
    diff = train_m * (train_r - pred_p)
    loss_p = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff), reduction='sum') + reg_losses
    loss_p.backward()
    return loss_p

def closure_f():
    optimizer_f.zero_grad()
    diff = train_m * (train_r - pred_f)
    loss_p = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff), reduction='sum') + reg_losses
    loss_p.backward()
    return loss_p

def loss_fn(pred, train_r, train_m, clip=True):
    diff = train_m * (train_r - pred)
    if clip:
        diff = torch.clamp(diff, min=1., max=5.)
    sqE = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff), reduction='sum')
    return sqE

In [491]:
# Init some variables
best_rmse_ep, best_mae_ep, best_ndcg_ep = 0, 0, 0
best_rmse, best_mae, best_ndcg = float("inf"), float("inf"), 0
time_cumulative = 0

# Initialize the model
pred_p, reg_losses = model_p(train_r)

# For optimizer_p:
for i in range(epoch_p):
  tic = time()
  optimizer_p.step(closure_p)
  pred_p, reg_losses = model_p(train_r)


  t = time() - tic
  time_cumulative += t

  # Calculate error, test_rmse, and train_rmse using PyTorch tensor operations
  error = (test_m * (pred_p.detach().numpy() - test_r) ** 2).sum() / test_m.sum()
  test_rmse = torch.sqrt(torch.tensor(error))

  error_train = (train_m * (pred_p - train_r) ** 2).sum() / train_m.sum()
  train_rmse = torch.sqrt(error_train)

  print('.-^-._' * 12)
  print('PRE-TRAINING')
  print('Epoch:', i+1, 'test rmse:', test_rmse.detach().numpy(), 'train rmse:', train_rmse.detach().numpy())
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)

# Now for optimizer_f

# Initialize the model
pred_f, reg_losses = model_f(train_r)

for i in range(epoch_f):
  tic = time()
  
  # Replace the optimizer_p.minimize() with a PyTorch training step

  optimizer_f.step(closure_f)

  # Replace 'pre' with the output of the PyTorch model
  
  pred_f, reg_losses = model_f(train_r)
  # pred_f = pred_f.detach().numpy()

  t = time() - tic
  time_cumulative += t
  # Following steps are basically copy of the original work
  error = (test_m * (pred_f.detach().numpy() - test_r) ** 2).sum() / test_m.sum()  # test error
  test_rmse = np.sqrt(torch.tensor(error))

  error_train = (train_m * (pred_f - train_r) ** 2).sum() / train_m.sum()  # train error
  train_rmse = np.sqrt(error_train.detach().numpy())

  test_mae = (test_m * np.abs((pred_f.detach().numpy() - test_r))).sum() / test_m.sum()
  
  train_mae = (train_m * torch.tensor(np.abs((pred_f - train_r).detach().numpy()))).sum() / train_m.sum()

  test_ndcg = call_ndcg(pred_f.detach().numpy(), test_r)
  train_ndcg = call_ndcg(pred_f.detach().numpy(), train_r)

  if test_rmse < best_rmse:
      best_rmse = test_rmse
      best_rmse_ep = i+1

  if test_mae < best_mae:
      best_mae = test_mae
      best_mae_ep = i+1

  if best_ndcg < test_ndcg:
      best_ndcg = test_ndcg
      best_ndcg_ep = i+1

  print('.-^-._' * 12)
  print('FINE-TUNING')
  print('Epoch:', i+1, 'test rmse:', test_rmse.detach().numpy(), 'test mae:', test_mae, 'test ndcg:', test_ndcg)
  print('Epoch:', i+1, 'train rmse:', train_rmse, 'train mae:', train_mae.detach().numpy(), 'train ndcg:', train_ndcg)
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)


.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 1 test rmse: 8.77899 train rmse: 8.921787
Time: 0.2527649402618408 seconds
Time cumulative: 0.2527649402618408 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 2 test rmse: 1.5280983 train rmse: 1.5699205
Time: 0.27286553382873535 seconds
Time cumulative: 0.5256304740905762 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 3 test rmse: 4.5751214 train rmse: 4.5672994
Time: 0.2767343521118164 seconds
Time cumulative: 0.8023648262023926 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 4 test rmse: 5.8203783 train rmse: 5.9227176


In [492]:
# Final result
print('Epoch:', best_rmse_ep, ' best rmse:', best_rmse.detach().numpy())
print('Epoch:', best_mae_ep, ' best mae:', best_mae)
print('Epoch:', best_ndcg_ep, ' best ndcg:', best_ndcg)

Epoch: 60  best rmse: 1.043332
Epoch: 60  best mae: 0.8311186
Epoch: 60  best ndcg: 0.8813742018256256


# Comparision with the original work:
More detailed writing will be given in the report.
To summarize, we have successfully reconstructed the algorithm with an deviation in ~0.1 RMSE.

# Sensitivity Analysis

In this section, I will be performing sensitivity analysis by adjusting hyperparameters.

Since there is a block for hyperparameters, such adjusting will be quite easy. Thus, here I will then post the results only.


Detailed anaylsis will be on the report.

## Base Line (Default Result)
Epoch: 60  best rmse: 1.038044

Epoch: 49  best mae: 0.82951796

Epoch: 60  best ndcg: 0.8719198874988237

## Epoch: 


Increasize number of epoch by 1.5:

Epoch: 90  best rmse: 0.97724164

Epoch: 90  best mae: 0.77384365

Epoch: 89  best ndcg: 0.8826587666740601



Increasize number of epoch by 2:

Epoch: 120  best rmse: 0.9544434

Epoch: 120  best mae: 0.75268626

Epoch: 120  best ndcg: 0.8872194687940944

## Number of layers:

Increase the size by 2 (4 layers):

Epoch: 60  best rmse: 1.0648588

Epoch: 47  best mae: 0.8483205

Epoch: 13  best ndcg: 0.85407570236797

## Hidden Dimension

Increase the size by 2 (1000):

Epoch: 60  best rmse: 1.0306457

Epoch: 60  best mae: 0.82486194

Epoch: 60  best ndcg: 0.8769577442754222

## Different Optimizer:

### SGD
SGD does not fit this problem. It will generate invalid results.

### RMSprop 

Epoch: 60  best rmse: 1.0599654

Epoch: 60  best mae: 0.8478273

Epoch: 1  best ndcg: 0.8426037939236917

## Learning rate (Adam Optimizer):

Increase by 10 times:

Epoch: 60  best rmse: 1.0374073

Epoch: 60  best mae: 0.8330115

Epoch: 60  best ndcg: 0.8731803489067932

Increase by 50 times:

Epoch: 60  best rmse: 1.043332

Epoch: 60  best mae: 0.8311186

Epoch: 60  best ndcg: 0.8813742018256256